In [12]:
from math import sqrt
P = 11
Point = tuple[int, int]

### Helper functions

In [13]:
def mod_mul(a: int, b: int, m: int = P):
	return ((a % m) * (b % m)) % m

In [14]:
def mod_pow(a: int, b: int, m: int = P) -> int:
	if b == 0:
		return 1
	r = mod_pow(a, b//2, m)
	r = (r * r) % m
	if b % 2:
		r = (r * a) % m
	return r

In [15]:
def mod_div(a: int, b: int, m: int = P):
	return mod_mul(a, mod_pow(b, m-2, m), m)

In [16]:
def generate_point(a: int, b: int):
	if (4 * (a**3) + 27 * (b**2)) == 0:
		raise ValueError("Enter other coefficients")
	
	x = 1
	y = 0
	while True:
		rhs = (x**3) + (a * x) + b
		y = int(sqrt(rhs))
		lhs = y**2
		if lhs == rhs:
			break
		x += 1

	return x, y

### ECC

In [17]:
class ECC:
	def __init__(self, a: int, b: int):
		self.a = a
		self.b = b

	def add(self, p1: Point, p2: Point, m: int = P):
		l = 0
		x1, y1 = p1
		x2, y2 = p2
		
		if p1 == p2:
			num = 3 * (x1**2) + a
			den = 2 * y1
		else:
			num = y2 - y1
			den = x2 - x1
		
		l = mod_div(num, den, m)
		x3 = ((l**2) - x1 - x2) % m
		y3 = (l * (x1 - x3) - y1) % m
		return x3, y3
	
	def mul(self, k: int, p: Point):
		temp = p
		while k != 1:
			temp = self.add(temp, p)
			k -= 1
		return temp

	def sub(self, p1: Point, p2: Point):
		x2, y2 = p2
		return self.add(p1, (x2, -y2))

### Main functions

In [18]:
def encrypt(ecc: ECC, plaintext: Point, public_key: Point, G: Point):
	k = 5
	return (
		ecc.mul(k, G),
		ecc.add(plaintext, ecc.mul(k, public_key))
	)

In [19]:
def decrypt(ecc: ECC, ciphertext: Point, private_key: int):
	x, y = ciphertext
	return ecc.sub(y, ecc.mul(private_key, x))

### I/O

In [20]:
a = int(input("Enter coefficient a:"))
b = int(input("Enter coefficient b:"))

ecc = ECC(a,b)
G = generate_point(a, b)
print(G)

(1, 2)


In [21]:
private_key = 5
public_key = ecc.mul(private_key, G)
print(public_key)

(1, 9)


In [22]:
plaintext = (3,4)
ciphertext = encrypt(ecc, plaintext, public_key, G)
print(ciphertext)

((1, 9), (8, 2))


In [23]:
decrypted_text = decrypt(ecc, ciphertext, private_key)
print(decrypted_text)

(3, 4)
